In [22]:
import pandas as pd
import numpy as np

from tsai.models.MINIROCKET import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.utils import shuffle

In [6]:
#Загружаем датасет
df = pd.read_csv('data/data_2022/only_zeya_fixed_actual_data.csv')
df

,Unnamed: 0,zeya-malayasazanka,zeya-svobodny,zeya-blagoveshensk,zeya-mazanovo,zeya-belogorye
0,2008-01-01,435.0,163.0,148.0,218.0,341.0
1,2008-01-02,441.0,162.0,151.0,222.0,350.0
2,2008-01-03,447.0,161.0,157.0,226.0,361.0
3,2008-01-04,447.0,154.0,163.0,230.0,367.0
4,2008-01-05,443.0,144.0,168.0,222.0,364.0
...,...,...,...,...,...,...
5254,2022-05-21,500.0,227.0,169.0,219.0,393.0
5255,2022-05-22,533.0,247.0,174.0,203.0,412.0
5256,2022-05-23,533.0,234.0,181.0,228.0,412.0
5257,2022-05-24,522.0,224.0,190.0,324.0,404.0


In [15]:
# X - Признаки(данные без благовещенска)
# y - Предикт (Только Благовещенск), т.к. предсказываем для благовещенска
def create_dataset(df_without_blagoveshensk, num_days=365):
    number_of_frames = df_without_blagoveshensk.to_numpy().shape[1]
    X = df_without_blagoveshensk.to_numpy().reshape(-1, number_of_frames, 1)
    array_for_concat = []
    for i in range(num_days):
        array_for_concat.append(X[i:X.shape[0]-(num_days - i)])
    new_X = np.concatenate(array_for_concat, axis=-1)
    y = df_without_blagoveshensk.to_numpy()[num_days:]
    
    return new_X, y

df_without_blagoveshensk = df[['zeya-malayasazanka', 'zeya-svobodny', 'zeya-mazanovo', 'zeya-belogorye']]
df_only_blagoveshensk = df[['zeya-blagoveshensk']]
X, y = create_dataset(df_without_blagoveshensk)
X.shape

(4894, 4, 365)

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=10, shuffle=False)
X_train.shape

(4884, 4, 365)

In [24]:
# Univariate regression with sklearn-type API

# X_train, y_train = shuffle(X_train, y_train)

rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
model = MiniRocketRegressor(scoring=rmse_scorer)
#print(model)
# timer.start(False)
model.fit(X_train, y_train)
# t = timer.stop()
y_pred = model.predict(X_valid)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
mae = mean_absolute_error(y_valid, y_pred)
print(f'valid rmse        : {rmse:.5f}')
print(f'valid mae        : {mae:.5f}')

valid rmse        : 37.75821
valid mae        : 29.73413
